In [7]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from exp.ProcessData import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import notebook
import PIL.Image
from PIL import  ImageDraw
import pydicom
import cv2
from torchvision import transforms
import torchvision.transforms.functional as TF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
params=json_to_parameters('config.json')
train_df=pd.read_csv(params.path.data+'train.csv')
test_df=pd.read_csv(params.path.data+'test.csv')

In [ ]:
class ImageDatasetProcess(Dataset):
    
    def __init__(self,file_path_in,file_path_out,df,transform=None,pre_out=''):
        super(ImageDatasetProcess, self).__init__()
        self.df=df
        self.file_list=list(file_path_in+df.image_name+'.jpg')
        self.file_list_out=list(file_path_out+pre_out+df.image_name+'.jpg')
        self.transform=transform
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,idx):
        img=self.transform(PIL.Image.open(self.file_list[idx]))
        img.save(self.file_list_out[idx])
        return torch.tensor(idx)


In [ ]:
train_df=pd.read_csv(params.path.data+'train.csv')
prepare_df(train_df)
train_df=train_df.reset_index(drop=True)
ds = ImageDatasetProcess(params.path.train_jpg,params.path.train_small,train_df,
                         transform=transforms.Compose([MyResize2(400,600)]))

dl = D.DataLoader(ds, batch_size=32,num_workers=18,shuffle=True)

for i in notebook.tqdm(dl):
    pass

In [ ]:
test_df=pd.read_csv(params.path.data+'test.csv')
prepare_df(test_df)
test_df=test_df.reset_index(drop=True)
ds = ImageDatasetProcess(params.path.test_jpg,params.path.test_jpg_small,test_df,
                         transform=transforms.Compose([MyResize2(400,600)]))
dl = D.DataLoader(ds, batch_size=32,num_workers=18,shuffle=True)
for i in notebook.tqdm(dl):
    pass

In [9]:
#Prepare ISIM2019 competition's data
isim2019_gt = pd.read_csv(params.path.ISIC_2019+'Training_GroundTruth.csv')
isim2019_meta = pd.read_csv(params.path.ISIC_2019+'Training_Metadata.csv')
isim2019_gt['diagnosis']=""

digdict={"MEL":"melanoma","NV":"nevus","BCC":"Basal cell carcinoma","AK":"Actinic keratosis","BKL":"Benign keratosis",
        "DF":"Dermatofibroma","VASC":"Vascular lesion","SCC":"Squamous cell carcinoma","UNK":"unknown"}
for key in digdict.keys():
    isim2019_gt['diagnosis']=np.where(isim2019_gt[key]>0,digdict[key],isim2019_gt['diagnosis'])
isim2019_df=isim2019_meta.merge(isim2019_gt[['image','diagnosis']],on='image').drop('lesion_id',1)
isim2019_df=isim2019_df.rename(columns={'image':'image_name','anatom_site_general':'anatom_site_general_challenge'})
isim2019_df['target']=(isim2019_df.diagnosis=='melanoma').astype(int)
isim2019_df['benign_malignant']=np.where(isim2019_df['diagnosis'].isin(["melanoma","Basal cell carcinoma","Squamous cell carcinoma"]),'malignant','benign')
isim2019_df.image_name='ISIM2019_'+isim2019_df.image_name
isim2019_df.to_csv(params.path.data+'isim2019.csv',index=False)

In [ ]:
ds = ImageDatasetProcess(params.path.train_ISIC_2019,params.path.train_small_ISIC_2019,isim2019_df,
                         transform=transforms.Compose([MyResize2(400,600)]),pre_out='ISIM2019_')
dl = D.DataLoader(ds, batch_size=32,num_workers=18,shuffle=True)
for i in notebook.tqdm(dl):
    pass